## Introduction
In order to do Data Science we have to be able to gather data. Oftentimes we get data from an API, sometimes from a database. This Jupyter notebook we'll guide you through a project that will involve getting data from an API, storing it in a relational database and performing some SQL queries to do some basic data analysis.

Our API of choice for this project will be [GitHub Jobs](https://jobs.github.com). Besides being simple to use, it could be a useful resource to job-hunt. 

## Assingment
- Get acquainted with the GitHub Jobs web interface and read through their API page.
- Import the Requests package and call the API with the parameters of your choice.
- Print the status code to make sure it went well.

In [39]:
import requests
resp = requests.get('https://jobs.github.com/positions.json?description=python&location=new+york')
print (resp.status_code)


200


## Assignment
- Import json and print the whole json response with the json.dumps() function. See what kind of data it contains.
- Get the dictionary of the first job and call the method keys() to have a better idea of the data at hand.
- Print some of the first job's content, including images and HTML. You might need to import the IPython package.

In [40]:
import json
#resp_dict = json.dumps(resp.content)
print ("\n")
#print ("******using .dumps() ********\n")
#print (type(resp_dict))
#print (resp_dict)
#print ("\n")
print ("******using .json() ********\n")
response_github = resp.json()
print (response_github)
print ("\n")





******using .json() ********

[{'id': '587a1d78-0428-11e7-8752-4c031c8e4a89', 'created_at': 'Wed Mar 08 17:57:18 UTC 2017', 'title': 'Senior Back-End Engineer, Stealth VC-Backed Fitness Technology Company', 'location': 'New York, NY', 'type': 'Full Time', 'description': '<p>About Us</p>\n\n<p>Our company is a stealth fit-tech concept being launched by an established leader in the NYC boutique fitness space with a team of experienced hardware and software engineers. We’re building a hybrid hardware/software product to bring high-quality, personalized fitness content into people’s homes across the country. Team includes experienced IOT engineers and developers, as well as top tier seed investors and advisors, with extensive background in launching premier consumer brands.</p>\n\n<p>Role</p>\n\n<p>Development of our core backend services and web applications, deployed on AWS, using PostgreSQL, Python and various front-end JS frameworks (React, etc.)\nMaintain an automated test suite to 

In [41]:
print ("********keys of the 1st job******\n")
print (resp.json()[0].keys())
print ("\n")

********keys of the 1st job******

dict_keys(['id', 'created_at', 'title', 'location', 'type', 'description', 'how_to_apply', 'company', 'company_url', 'company_logo', 'url'])




In [42]:
print ("********content of the 1st job******\n")

print (resp.json()[0])

********content of the 1st job******

{'id': '587a1d78-0428-11e7-8752-4c031c8e4a89', 'created_at': 'Wed Mar 08 17:57:18 UTC 2017', 'title': 'Senior Back-End Engineer, Stealth VC-Backed Fitness Technology Company', 'location': 'New York, NY', 'type': 'Full Time', 'description': '<p>About Us</p>\n\n<p>Our company is a stealth fit-tech concept being launched by an established leader in the NYC boutique fitness space with a team of experienced hardware and software engineers. We’re building a hybrid hardware/software product to bring high-quality, personalized fitness content into people’s homes across the country. Team includes experienced IOT engineers and developers, as well as top tier seed investors and advisors, with extensive background in launching premier consumer brands.</p>\n\n<p>Role</p>\n\n<p>Development of our core backend services and web applications, deployed on AWS, using PostgreSQL, Python and various front-end JS frameworks (React, etc.)\nMaintain an automated test suit

In [43]:
from IPython.display import Image
image_url = resp.json()[0]['company_logo']
Image(url=image_url) 

In [44]:
from IPython.core.display import display, HTML
display(HTML(resp.json()[0]['how_to_apply']))

## Assignment
- Import SQLAlchemy and configure a SQLite database.
- Create 4 tables: company, job, location, github_metadata. Pay sppecial attention on primary and foreign keys as well as unique and non-nullable columns.
- Make sure you store 'created_at' as a datetime and not a string.

In [45]:
import sqlalchemy
from sqlalchemy import * #create_engine, Table, MetaData
db = create_engine('sqlite:///:memory:', echo=True)

metadata = MetaData()
'''
jobs = Table('', Base.metadata,
    Column('job_id', Integer, ForeignKey('jobs.id'))
    
)
'''
company_table = Table('company', metadata,
                      Column('id', Integer, primary_key=True),
                      Column('name', String, nullable=False),
                      Column('url', String, unique=True),
                      Column('logo_url', String))

job_table = Table('job', metadata,
                  Column('id', Integer, primary_key=True),
                  Column('company_id', None, ForeignKey('company.id'), nullable=False),
                  Column('type_', Integer),
                  Column('title', String, nullable=False),
                  Column('description', String),
                  Column('how_to_apply', String))

location_table = Table('location', metadata,
                      Column('job_id', None, ForeignKey('job.id'), primary_key=True),
                      Column('description', String),
                      Column('remote_ok', Integer))

github_metadata_table = Table('github_metadata', metadata,
                              Column('job_id', None, ForeignKey('job.id'), primary_key=True),
                              Column('github_id', String, unique=True),
                              Column('url', String, nullable=False),
                              Column('created_at', DateTime, nullable=False))

metadata.create_all(db)
conn = db.connect()


2017-03-10 21:15:33,215 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-03-10 21:15:33,216 INFO sqlalchemy.engine.base.Engine ()
2017-03-10 21:15:33,219 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-03-10 21:15:33,221 INFO sqlalchemy.engine.base.Engine ()
2017-03-10 21:15:33,222 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("company")
2017-03-10 21:15:33,223 INFO sqlalchemy.engine.base.Engine ()
2017-03-10 21:15:33,225 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("job")
2017-03-10 21:15:33,226 INFO sqlalchemy.engine.base.Engine ()
2017-03-10 21:15:33,228 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("location")
2017-03-10 21:15:33,228 INFO sqlalchemy.engine.base.Engine ()
2017-03-10 21:15:33,229 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("github_metadata")
2017-03-10 21:15:33,230 INFO sqlalchemy.engine.base.Engine ()
2017-03-10 21:15:33,232 INFO sqla

## Assignment
- Store the data you got from the API request in the database.
- Make sure you create a transaction (i.e., if an error occurs while saving a job's location data the database should rollback on all statements regarding that job).

In [46]:
from enum import Enum

class JobType(Enum):
    full_time = 1
    part_time = 2

    @staticmethod
    def from_string(string):
        if string.lower() == 'full time' or string.lower() == 'full-time':
            return JobType.full_time
        elif string.lower() == 'part time' or string.lower() == 'part-time':
            return JobType.part_time
        else:
            raise RuntimeError('JobType not supported: {}'.format(string))

In [47]:
def add_company(name, url, logo_url, connection=conn):

    insert = company_table.insert().values(name=name, url=url, logo_url=logo_url)
    result = connection.execute(insert)
    return result.inserted_primary_key[0]

def add_job(company_id, type_str, title, description, how_to_apply, connection=conn):

    insert = job_table.insert().values(company_id=company_id, 
                                       type_=JobType.from_string(type_str).value, 
                                       title=title, description=description, how_to_apply=how_to_apply) 
    result = connection.execute(insert)
    return result.inserted_primary_key[0]

def add_location(job_id, description, connection=conn):

    insert = location_table.insert().values(job_id=job_id, description=description, 
                                       remote_ok=int('remote' in description.lower() 
                                                     or 'telecommute' in description.lower())) 
    result = connection.execute(insert)
    return result.inserted_primary_key[0]

import datetime
created_at_pattern = '%a %b %d %X %Z %Y'

def add_github_metadata(job_id, github_id, url, created_at_str, connection=conn):

    insert = github_metadata_table.insert().values(job_id=job_id, github_id=github_id, url=url,
                                       created_at=datetime.datetime.strptime(created_at_str, created_at_pattern))
    result = connection.execute(insert)
    return result.inserted_primary_key[0]

In [48]:
from sqlalchemy.sql import select

def select_or_insert_company(name, url, logo_url, connection=conn):

    s = select([company_table]).where(company_table.c.url == url)
    result = connection.execute(s).fetchone()
    if result is not None:
        return result['id']
    else:
        new_id = add_company(name, url, logo_url, connection)
        return new_id

In [49]:
def select_or_insert_job(company_name, company_url, company_logo_url, 
                         github_id, github_url, github_created_at_str,
                         type_str, title, description, how_to_apply,
                         location_description):
    """
    Returns the job id either by selecting from or inserting in job_table.
    
    This function might also insert new location, github_metadata and company entries.
    """
    s = select([github_metadata_table]).where(github_metadata_table.c.github_id == github_id)
    result = conn.execute(s).fetchone()
    if result is not None:
        return result['job_id']
    else:
        with db.begin() as transaction:
            company_id = select_or_insert_company(company_name, company_url, company_logo_url, transaction)
            job_id = add_job(company_id, type_str, title, description, how_to_apply, transaction)
            add_github_metadata(job_id, github_id, github_url, github_created_at_str, transaction)
            add_location(job_id, location_description, transaction)
            
        return job_id
    

In [50]:
print ("what's json response..", response_github)

what's json response.. [{'id': '587a1d78-0428-11e7-8752-4c031c8e4a89', 'created_at': 'Wed Mar 08 17:57:18 UTC 2017', 'title': 'Senior Back-End Engineer, Stealth VC-Backed Fitness Technology Company', 'location': 'New York, NY', 'type': 'Full Time', 'description': '<p>About Us</p>\n\n<p>Our company is a stealth fit-tech concept being launched by an established leader in the NYC boutique fitness space with a team of experienced hardware and software engineers. We’re building a hybrid hardware/software product to bring high-quality, personalized fitness content into people’s homes across the country. Team includes experienced IOT engineers and developers, as well as top tier seed investors and advisors, with extensive background in launching premier consumer brands.</p>\n\n<p>Role</p>\n\n<p>Development of our core backend services and web applications, deployed on AWS, using PostgreSQL, Python and various front-end JS frameworks (React, etc.)\nMaintain an automated test suite to ensure th

In [58]:
for job in response_github:
    print ("ID", select_or_insert_job(
        job['company'],
        job['company_url'],
        job['company_logo'],
        job['id'],
        job['url'],
        job['created_at'],
        job['type'],
        job['title'],
        job['description'],
        job['how_to_apply'],
        job['location']
    ))


2017-03-10 21:19:55,584 INFO sqlalchemy.engine.base.Engine SELECT github_metadata.job_id, github_metadata.github_id, github_metadata.url, github_metadata.created_at 
FROM github_metadata 
WHERE github_metadata.github_id = ?
2017-03-10 21:19:55,586 INFO sqlalchemy.engine.base.Engine ('587a1d78-0428-11e7-8752-4c031c8e4a89',)
ID 1
2017-03-10 21:19:55,588 INFO sqlalchemy.engine.base.Engine SELECT github_metadata.job_id, github_metadata.github_id, github_metadata.url, github_metadata.created_at 
FROM github_metadata 
WHERE github_metadata.github_id = ?
2017-03-10 21:19:55,589 INFO sqlalchemy.engine.base.Engine ('0b6b0a38-0428-11e7-87e3-69c380153781',)
ID 2
2017-03-10 21:19:55,591 INFO sqlalchemy.engine.base.Engine SELECT github_metadata.job_id, github_metadata.github_id, github_metadata.url, github_metadata.created_at 
FROM github_metadata 
WHERE github_metadata.github_id = ?
2017-03-10 21:19:55,592 INFO sqlalchemy.engine.base.Engine ('529b9d4c-0373-11e7-93ef-382981ce9f73',)
ID 3
2017-03-10

## Assignment
- Execute simple SQL queries in each table and make sure everything looks as was intended.
- Create queries the following queries:
    - Find out how many jobs have the phrase "Data Science" and how many have the word "Python" in the description
    - Find out how many were created in the past 7 days
    - Find out how are "Remote Ok" and which companies those are with

**Find out how many jobs have the phrase "Data Science" and how many have the word "Python" in the description**

In [52]:
s = text("SELECT COUNT() "
         "FROM job")
total, = conn.execute(s).fetchone()

print ("what is total", total)

s = text("SELECT COUNT() "
         "FROM job "
         "WHERE description LIKE '%python%'")
python, = conn.execute(s).fetchone()

print ("there are {0} jobs with 'Data Science' term, and {1} percent of them have 'Python' in the description".format(total, 100*python/total))


2017-03-10 21:15:33,507 INFO sqlalchemy.engine.base.Engine SELECT COUNT() FROM job
2017-03-10 21:15:33,509 INFO sqlalchemy.engine.base.Engine ()
what is total 11
2017-03-10 21:15:33,511 INFO sqlalchemy.engine.base.Engine SELECT COUNT() FROM job WHERE description LIKE '%python%'
2017-03-10 21:15:33,512 INFO sqlalchemy.engine.base.Engine ()
there are 11 jobs with 'Data Science' term, and 100.0 percent of them have 'Python' in the description


**Find out how many were created in the past 7 days**

In [53]:
from datetime import datetime, timedelta

s = text("SELECT COUNT() "
         "FROM github_metadata "
         "WHERE created_at >= :week_ago")
last_week, = conn.execute(s, week_ago=datetime.today() - timedelta(days=7)).fetchone()

# # "First argument: {0}, second one: {1}".format(47,11) 
print ("{0} percent of them have been created in the past 7 days".format(100 * last_week/total))


2017-03-10 21:15:33,519 INFO sqlalchemy.engine.base.Engine SELECT COUNT() FROM github_metadata WHERE created_at >= ?
2017-03-10 21:15:33,521 INFO sqlalchemy.engine.base.Engine (datetime.datetime(2017, 3, 3, 21, 15, 33, 519593),)
36.36363636363637 percent of them have been created in the past 7 days


**Find out how are "Remote Ok" and which companies those are with**

In [56]:
s = text("SELECT COUNT() "
         "FROM job "
         "JOIN location ON job.id = location.job_id "
         "WHERE remote_ok = :true")
remote_jobs, = conn.execute(s, true=int(True)).fetchone()
print ("remote_jobs", remote_jobs)
if remote_jobs > 0:
    s = text("SELECT name "
             "FROM company "
             "JOIN job ON company.id = job.company_id "
             "JOIN location ON job.id = location.job_id "
             "WHERE location.remote_ok = :true")
    remote_companies = conn.execute(s, true=int(True)).fetchall()
    
    #if len(remote_companies)==1:
        
    
    print ("The number of company(s) ok with remote job is {0} and the names are: {1}".format(len(remote_companies), remote_companies))

    for company in remote_companies:
        print ("company: ", company)

    print(message.format(remote_jobs, " ;".join(str(company) for company, in remote_companies)))

2017-03-10 21:17:21,214 INFO sqlalchemy.engine.base.Engine SELECT COUNT() FROM job JOIN location ON job.id = location.job_id WHERE remote_ok = ?
2017-03-10 21:17:21,216 INFO sqlalchemy.engine.base.Engine (1,)
remote_jobs 0
